In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("MultilayerPerceptronClassifier").getOrCreate()

23/12/27 14:35:54 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/27 14:35:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 14:35:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/27 14:36:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/27 14:36:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
iris = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/iris.csv", header=True, inferSchema=True,
                      sep=",")
iris.show()

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

In [5]:
from pyspark.ml.feature import VectorAssembler

asb = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

irisas = asb.transform(iris)
irisas.show()

+-----------+----------+-----------+----------+-----------+-----------------+
|sepallength|sepalwidth|petallength|petalwidth|      class|     independente|
+-----------+----------+-----------+----------+-----------+-----------------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|[4.4,2.9,1.4,0.2]|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|[4.9,

In [6]:
from pyspark.ml.feature import StringIndexer

ind = StringIndexer(inputCol="class", outputCol="dependente")

irisas = ind.fit(irisas).transform(irisas)
irisas.show()

+-----------+----------+-----------+----------+-----------+-----------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|     independente|dependente|
+-----------+----------+-----------+----------+-----------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|       0.0|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|       0.0|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|       0.0|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|       0.0|
|        4.4|       2

In [7]:
irisTreino, irisTeste = irisas.randomSplit([0.7, 0.3])

print(irisTreino.count())
print(irisTeste.count())

90


60


In [8]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

mlp = MultilayerPerceptronClassifier(maxIter=10, layers=[4,5,4,3], featuresCol="independente", labelCol="dependente")
modelo = mlp.fit(irisTreino)

In [10]:
previsao = modelo.transform(irisTeste)
previsao.select("dependente", "prediction").show(25)

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       2.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 25 rows



In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)

print(f"Acurácia: {acuracia}")

Acurácia: 0.65


In [12]:
print(modelo.getMaxIter())
print(modelo.getLayers())
print(modelo.getStepSize())

10
[4, 5, 4, 3]
0.03


In [13]:
parunico = {modelo.maxIter: 1000}

In [14]:
modelo = mlp.fit(irisTreino, parunico)

In [15]:
print(modelo.getMaxIter())
print(modelo.getLayers())
print(modelo.getStepSize())

1000
[4, 5, 4, 3]
0.03


In [16]:
previsao = modelo.transform(irisTeste)
previsao.select("dependente", "prediction").show(25)

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       2.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 25 rows



In [17]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)

print(f"Acurácia: {acuracia}")

Acurácia: 0.9333333333333333
